In [1]:
import pandas as pd
import requests
import psycopg2
from datetime import datetime, timedelta
import json
import time
import string
import uuid
import os
import sys

In [7]:
# Set the working directory explicitly
os.chdir('C:/Users/A/Documents/HS23_DWL/Git/DWLadies')

# Verify the current working directory
print("Current Working Directory:", os.getcwd())
# Print the content of the working directory
print("Directory Contents:", os.listdir())

sys.path.insert(0, os.getcwd())
print(sys.path)

# Try to import the Config module
try:
    from myconfig.config import config
    
    config_data = config.get_config()
    
    hostname = config_data["hostname"]
    port = config_data["port"]
    database = config_data["database"]
    username = config_data["username"]
    password = config_data["password"]
    PERSONAL_ACCESS_TOKEN = config_data["PERSONAL_ACCESS_TOKEN"]
    
    print("Config module imported successfully.")
    
except ImportError as e:
    print("Error importing Config module:", e)

Current Working Directory: C:\Users\A\Documents\HS23_DWL\Git\DWLadies
Directory Contents: ['.git', '.ipynb_checkpoints', 'ConnectDBtoPostgres.txt', 'Holiday_API_Connection.ipynb', 'Holiday_Lambda_Function.ipynb', 'install_lambda_layer_python_3.9.txt', 'myconfig', 'Package', 'Package1', 'README.md', 'Weather_API_Connection.ipynb', 'Weather_Lambda_Function.ipynb', '__init__.ipynb']
['C:\\Users\\A\\Documents\\HS23_DWL\\Git\\DWLadies', 'C:\\Users\\A\\Documents\\HS23_DWL\\Git\\DWLadies', 'C:\\Users\\A\\Documents\\HS23_DWL\\Git\\DWLadies', 'C:\\Users\\A\\Documents\\HS23_DWL\\Git\\DWLadies', 'C:\\Users\\A\\Documents\\HS23_DWL\\Git\\DWLadies', 'C:\\Users\\A\\anaconda3\\python39.zip', 'C:\\Users\\A\\anaconda3\\DLLs', 'C:\\Users\\A\\anaconda3\\lib', 'C:\\Users\\A\\anaconda3', '', 'C:\\Users\\A\\anaconda3\\lib\\site-packages', 'C:\\Users\\A\\anaconda3\\lib\\site-packages\\win32', 'C:\\Users\\A\\anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\Users\\A\\anaconda3\\lib\\site-packages\\Pythonwin']
E

In [4]:
# Websites where the Lambda Function looking for new commits:

# Holiday Data CH:
# https://github.com/openpotato/openholidaysapi.data/tree/main/src/ch/holidays

# Holiday Data FR:
# https://github.com/openpotato/openholidaysapi.data/tree/main/src/fr/holidays

# Holiday Data DE:
# https://github.com/openpotato/openholidaysapi.data/tree/main/src/de/holidays

In [5]:
# connection to PostgresDB

def connect_db(hostname, port, database, username, password):
    try: 
        conn = psycopg2.connect(
            host=hostname,
            port=port,
            dbname=database,
            user=username,
            password=password)
    except psycopg2.Error as e: 
        print("Error: Could not get curser to the Database")
        print(e)

    # Auto commit is very important
    conn.set_session(autocommit=True)
    
    print("connection successful")

In [6]:
# Delete the data where the start date is equal to or more than today's year

def delete_future_data():
    delete_future_holidays = """DELETE FROM holidays
    WHERE CAST(start_date AS DATE) >= CURRENT_DATE;"""

    # Connect cursor
    cur = conn.cursor()

    # Execute data deletion
    cur.execute(delete_future_holidays)
    
    print("current and future data deleted")
    

In [7]:
# Create function: data insert public holidays table

def insert_holidays(data):
    for item in data:
        id_holiday = item["id"]
        isoCode_holiday = country,
        start_date = item["startDate"]
        end_date = item["endDate"]
        type_holiday = item["type"]
        name_holiday = [entry["text"] for entry in item["name"] if entry ["language"] == "EN"]
        nationwide = item["nationwide"]
        subdivisions_holiday = item.get("code", [])
        if "subdivisions" in item:
            subdivisions_holiday = [entry["code"] for entry in item["subdivisions"]]
        else:
            subdivisions_holiday = []
        
        cur.execute(
                "INSERT INTO holidays (id_holiday, isoCode_holiday, start_date, end_date, type_holiday, name_holiday, nationwide, subdivisions_holiday) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)",
                (id_holiday, isoCode_holiday, start_date, end_date, type_holiday, name_holiday, nationwide, subdivisions_holiday)
            )

In [8]:
# Create function reload data from OpenholidaysAPI into Postgres database "holidays" table

def load_holidays():
    
    # OpenholidayAPI parameters:
    API_URL = "https://openholidaysapi.org/"
    typ = ["PublicHolidays?","SchoolHolidays?"]
    country_code = "countryIsoCode=" + countries[country]
    language_code = "&languageIsoCode=EN"
    
    # Creating the OpenholidayAPI urls:
    for y in years:
        date_from = "&validFrom=" + str(y) + today.strftime('-%m-%d')
        date_to = "&validTo=" + str(y+1) + yesterday.strftime('-%m-%d')
        for i in range(0,2):
            HOLIDAY_API = API_URL + typ[i] + country_code + language_code + date_from + date_to
            print(HOLIDAY_API)
            response = requests.get(HOLIDAY_API)
            data = response.json()

            # Check if the list is empty, if empty do not insert
            if not data:  
                print("The holidaylist for is empty. Continuing...")
            else:
                # Connect cursor
                cur = conn.cursor()

                try:
                    insert_holidays(data)
                    print("successfully finished")

                except psycopg2.Error as e: 
                    print("Error: Inserting Rows")
                    print (e)


In [10]:
# Given neighbouring countries of Basel -> we are interested only Switzerland, Germany and France
countries =["CH","DE","FR"]

# Dates current year & future (for the lambda function)
today = datetime.now().date()
yesterday = datetime.now().date()-timedelta(1)
year =  datetime.today().year
years = list(range(year, year + 10))
print("Today`s date:", today, "\n ***** \n")

# test if update process works:
#test = datetime.now().date()-timedelta(33)
#print("Test date:", test, "\n ***** \n")

# GitHub API URL pointing to the OpenholidaysAPI repository
repo_url_ch = "https://api.github.com/repos/openpotato/openholidaysapi.data/commits?path=src/ch/holidays&per_page=1"
repo_url_de = "https://api.github.com/repos/openpotato/openholidaysapi.data/commits?path=src/de/holidays&per_page=1"
repo_url_fr = "https://api.github.com/repos/openpotato/openholidaysapi.data/commits?path=src/fr/holidays&per_page=1"

# Check every day, if there was a new commit in one of the holiday libraries on HolidayAPI github data:
for country in range(len(countries)):
    repo_url = "https://api.github.com/repos/openpotato/openholidaysapi.data/commits?path=src/"+ countries[country].lower()+"/holidays&per_page=1"
    
    # Make a GET request to the GitHub API
    headers = {"Authorization": str(PERSONAL_ACCESS_TOKEN)}
    response = requests.get(repo_url, headers=headers)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        commit_info = response.json()

        # Get the commit date
        commit_date_str = commit_info[0]['commit']['committer']['date']
        commit_date = datetime.strptime(commit_date_str, "%Y-%m-%dT%H:%M:%SZ").date()

        # Compare dates and print if they are the same
        #if commit_date < test:
        if commit_date < today:
            print(countries[country], "Commit's date:", commit_date, "\n ***** \n")
        else:
            print (countries[country],"updates found process started:")
            connect_db(hostname, port, database, username, password)
            delete_future_data()
            load_holidays()
            print("update finished \n ***** \n")
            # Close the cursor and connection when done
            cur.close()
            conn.close()
    
    else:
        print("Error: Failed to fetch data from GitHub API. Status code:", response.status_code)

Today`s date: 2023-11-19 
 ***** 

CH Commit's date: 2023-02-02 
 ***** 

DE Commit's date: 2023-10-17 
 ***** 

FR Commit's date: 2023-02-22 
 ***** 



In [11]:
# Creating Lambda handler function in AWS

#def lambda_handler(event, context):
    # connect to the API
    # transform it
    # load it into a database
    # max time limit 15 minutes
    
    
    # TODO implement
#    return {
#        'statusCode': 200,
#        'body': json.dumps('Hello from DW!')
#    }
current_datetime = datetime.now()
print(current_datetime)

2023-11-19 16:03:47.559282
